## Instituto Federal da Paraíba
## Disciplina: Mineração de Dados
## Professor: Paulo Ribeiro
## Alunos: Filipe Salomão, Hércules de Sousa e Matheus Alves
##
## Projeto: Classificação de Gêneros Musicais

Importanto bibliotecas necessárias:

In [1]:
import librosa
import numpy as np
import os
import pandas as pd
from joblib import load
from scipy.stats import kurtosis
from scipy.stats import skew
import warnings

Reutilizando a função **_get_features_**:

In [2]:
def get_features(y, sr, n_fft=1024, hop_length=512):
    features = {}

    features['spectral_centroid'] = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_rolloff'] = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['zero_crossing_rate'] = librosa.feature.zero_crossing_rate(y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['rms'] = librosa.feature.rms(y=y, frame_length=n_fft, hop_length=hop_length).ravel()
    features['onset_strength'] = librosa.onset.onset_strength(y=y, sr=sr).ravel()
    features['spectral_contrast'] = librosa.feature.spectral_contrast(y=y, sr=sr).ravel()
    features['spectral_bandwidth'] = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length).ravel()
    features['spectral_flatness'] = librosa.feature.spectral_flatness(y=y, n_fft=n_fft, hop_length=hop_length).ravel()

    mfcc = librosa.feature.mfcc(y=y, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)

    for index, v_mfcc in enumerate(mfcc):
        features['mfcc_{}'.format(index)] = v_mfcc.ravel()

    def get_moments(descriptors):
        result = {}

        for k, v in descriptors.items():
            result['{}_max'.format(k)] = np.max(v)
            result['{}_min'.format(k)] = np.min(v)
            result['{}_mean'.format(k)] = np.mean(v)
            result['{}_std'.format(k)] = np.std(v)
            result['{}_kurtosis'.format(k)] = kurtosis(v)
            result['{}_skew'.format(k)] = skew(v)

        return result

    additional_features = get_moments(features)
    additional_features['tempo'] = librosa.feature.tempo(y=y, sr=sr)[0]

    return additional_features

Diretório onde estão as músicas a serem testadas:

In [3]:
music_directory = 'music'

Carregando todos os modelos para testá-los:

In [4]:
pipe_lr = load('./models/pipe_lr.joblib')

In [5]:
pipe_dt = load('./models/pipe_dt.joblib')

In [6]:
pipe_rf = load('./models/pipe_rf.joblib')

In [7]:
pipe_svm = load('./models/pipe_svm.joblib')

In [8]:
pipe_knn = load('./models/pipe_knn.joblib')

In [9]:
pipe_nb = load('./models/pipe_nb.joblib')

A função **_test_model_** recebe um pipeline de modelo, um diretório contendo músicas e uma lista de gêneros musicais. Ela realiza predições das músicas de cada gênero utilizando o modelo fornecido e imprime os resultados das predições. Durante a execução, são ignorados os avisos gerados.

In [10]:
def test_model(pipe, directory):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')

        results = {}

        genres = os.listdir(directory)

        for genre in genres:
            genre_results = []

            print(f'Realizando predições de músicas de {genre}...\n')

            genre_directory = os.path.join(directory, genre)
            genre_music_files = os.listdir(genre_directory)

            for music in genre_music_files:
                signal, sr = librosa.load(os.path.join(genre_directory, music))
                features = get_features(signal, sr)
                song = pd.DataFrame([features])

                pred = pipe.predict(song)[0]

                genre_results.append((genre, music, pred))

                print(f'Gênero {genre}: {music} foi predito como música de {pred}')

            results[genre] = genre_results
            
            print()

In [11]:
test_model(pipe_lr, music_directory)

Realizando predições de músicas de forró...


Realizando predições de músicas de funk...


Realizando predições de músicas de jazz...


Realizando predições de músicas de rap brasileiro...


Realizando predições de músicas de rap internacional...


Realizando predições de músicas de rock brasileiro...

Gênero rock brasileiro: CPM 22 - Um Minuto Para O Fim Do Mundo.mp3 foi predito como música de rock
Gênero rock brasileiro: Engenheiros do Hawaii - Até O Fim.mp3 foi predito como música de rock
Gênero rock brasileiro: NX Zero - Além De Mim.mp3 foi predito como música de forró
Gênero rock brasileiro: Pitty - Máscara.mp3 foi predito como música de rock
Gênero rock brasileiro: Tihuana - Tropa De Elite.mp3 foi predito como música de rap

Realizando predições de músicas de rock internacional...


Realizando predições de músicas de samba...


Realizando predições de músicas de sertanejo...




In [12]:
test_model(pipe_dt, music_directory)

Realizando predições de músicas de forró...


Realizando predições de músicas de funk...


Realizando predições de músicas de jazz...


Realizando predições de músicas de rap brasileiro...


Realizando predições de músicas de rap internacional...


Realizando predições de músicas de rock brasileiro...

Gênero rock brasileiro: CPM 22 - Um Minuto Para O Fim Do Mundo.mp3 foi predito como música de rock
Gênero rock brasileiro: Engenheiros do Hawaii - Até O Fim.mp3 foi predito como música de rock
Gênero rock brasileiro: NX Zero - Além De Mim.mp3 foi predito como música de rock
Gênero rock brasileiro: Pitty - Máscara.mp3 foi predito como música de rock
Gênero rock brasileiro: Tihuana - Tropa De Elite.mp3 foi predito como música de rap

Realizando predições de músicas de rock internacional...


Realizando predições de músicas de samba...


Realizando predições de músicas de sertanejo...




In [13]:
test_model(pipe_rf, music_directory)

Realizando predições de músicas de forró...


Realizando predições de músicas de funk...


Realizando predições de músicas de jazz...


Realizando predições de músicas de rap brasileiro...


Realizando predições de músicas de rap internacional...


Realizando predições de músicas de rock brasileiro...

Gênero rock brasileiro: CPM 22 - Um Minuto Para O Fim Do Mundo.mp3 foi predito como música de rock
Gênero rock brasileiro: Engenheiros do Hawaii - Até O Fim.mp3 foi predito como música de forró
Gênero rock brasileiro: NX Zero - Além De Mim.mp3 foi predito como música de rock
Gênero rock brasileiro: Pitty - Máscara.mp3 foi predito como música de rock
Gênero rock brasileiro: Tihuana - Tropa De Elite.mp3 foi predito como música de rock

Realizando predições de músicas de rock internacional...


Realizando predições de músicas de samba...


Realizando predições de músicas de sertanejo...




In [14]:
test_model(pipe_svm, music_directory)

Realizando predições de músicas de forró...


Realizando predições de músicas de funk...


Realizando predições de músicas de jazz...


Realizando predições de músicas de rap brasileiro...


Realizando predições de músicas de rap internacional...


Realizando predições de músicas de rock brasileiro...

Gênero rock brasileiro: CPM 22 - Um Minuto Para O Fim Do Mundo.mp3 foi predito como música de rock
Gênero rock brasileiro: Engenheiros do Hawaii - Até O Fim.mp3 foi predito como música de rock
Gênero rock brasileiro: NX Zero - Além De Mim.mp3 foi predito como música de rock
Gênero rock brasileiro: Pitty - Máscara.mp3 foi predito como música de rock
Gênero rock brasileiro: Tihuana - Tropa De Elite.mp3 foi predito como música de rap

Realizando predições de músicas de rock internacional...


Realizando predições de músicas de samba...


Realizando predições de músicas de sertanejo...




In [15]:
test_model(pipe_knn, music_directory)

Realizando predições de músicas de forró...


Realizando predições de músicas de funk...


Realizando predições de músicas de jazz...


Realizando predições de músicas de rap brasileiro...


Realizando predições de músicas de rap internacional...


Realizando predições de músicas de rock brasileiro...

Gênero rock brasileiro: CPM 22 - Um Minuto Para O Fim Do Mundo.mp3 foi predito como música de rock
Gênero rock brasileiro: Engenheiros do Hawaii - Até O Fim.mp3 foi predito como música de rap
Gênero rock brasileiro: NX Zero - Além De Mim.mp3 foi predito como música de rock
Gênero rock brasileiro: Pitty - Máscara.mp3 foi predito como música de forró
Gênero rock brasileiro: Tihuana - Tropa De Elite.mp3 foi predito como música de rap

Realizando predições de músicas de rock internacional...


Realizando predições de músicas de samba...


Realizando predições de músicas de sertanejo...




In [16]:
test_model(pipe_nb, music_directory)

Realizando predições de músicas de forró...


Realizando predições de músicas de funk...


Realizando predições de músicas de jazz...


Realizando predições de músicas de rap brasileiro...


Realizando predições de músicas de rap internacional...


Realizando predições de músicas de rock brasileiro...

Gênero rock brasileiro: CPM 22 - Um Minuto Para O Fim Do Mundo.mp3 foi predito como música de rock
Gênero rock brasileiro: Engenheiros do Hawaii - Até O Fim.mp3 foi predito como música de rap
Gênero rock brasileiro: NX Zero - Além De Mim.mp3 foi predito como música de rock
Gênero rock brasileiro: Pitty - Máscara.mp3 foi predito como música de forró
Gênero rock brasileiro: Tihuana - Tropa De Elite.mp3 foi predito como música de rap

Realizando predições de músicas de rock internacional...


Realizando predições de músicas de samba...


Realizando predições de músicas de sertanejo...


